In [ ]:

NUM_PROC_MAP = 150 # expect 30 min with single process
PUSH_TO_HF = False

DEBUG = True
EPOCHS = 10
HF_USERNAME = None

import os

from dotenv import load_dotenv
load_dotenv()
import datasets
from datasets import DatasetDict
from datasets import load_dataset
import datasets
import torch
import json
from huggingface_hub import HfApi

import numpy as np

from scipy.stats import norm
from scipy.stats import f
from scipy.stats import lognorm
import plotting
import matplotlib.pyplot as plt
import json
import os
import pandas as pd
import torch
import datasets
from datasets import load_dataset




DEBUG = True

import torch
from scipy.stats import kendalltau
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np


from baseline_curricula import validate_training_duration_limitation
from itertools import product, chain
import config 
from util import get_curriculum

jobs =[(dataset, curriculum,model_type) for dataset, model_type, curriculum in product(config.datasets, config.model_types, config.baseline_curricula)]
jobs.extend([(dataset, model_type + curriculum,model_type) for dataset, model_type, curriculum  in (product(config.datasets, config.model_types, config.influence_curricula))])


In [ ]:
from transformers import RobertaForMaskedLM, DataCollatorForLanguageModeling, LlamaForCausalLM, LlamaTokenizerFast
from transformers import RobertaTokenizerFast


In [ ]:
model_path = "models/stratified_10m_curriculum_roberta_roberta_influence_incr_bins_lognorm_babylm"

In [ ]:
model = RobertaForMaskedLM.from_pretrained(model_path)
tokenizer = RobertaTokenizerFast.from_pretrained(model_path, max_len=512)


In [ ]:
model.push_to_hub(os.path.basename(model_path), revision="main")
tokenizer.push_to_hub(os.path.basename(model_path), revision="main")


In [ ]:
" ".join(['"chck_'+ a+'"' for a in ["1M","2M","3M","4M","5M","6M", "7M", "8M", "9M", "10M", "20M", "30M", "40M", "50M","60M", "70M", "80M", "90M"]])

In [ ]:
len([(checkpoint, name) for checkpoint, name in zip(sorted([a for a in os.listdir(os.path.join(model_path)) if "checkpoint" in a],key=lambda x: int(x.split("-")[-1])),["1M","2M","3M","4M","5M","6M", "7M", "8M", "9M", "10M", "20M", "30M", "40M", "50M","60M", "70M", "80M", "90M"])])

In [ ]:
for checkpoint_dir, name in [(checkpoint, name) for checkpoint, name in zip(sorted([a for a in os.listdir(os.path.join(model_path)) if "checkpoint" in a],key=lambda x: int(x.split("-")[-1])),["1M","2M","3M","4M","5M","6M", "7M", "8M", "9M", "10M", "20M", "30M", "40M", "50M","60M", "70M", "80M", "90M"])][0:11]:
    model = RobertaForMaskedLM.from_pretrained(os.path.join(model_path,checkpoint_dir))
    model.push_to_hub(os.path.basename(model_path), revision="chck_"+name)
    tokenizer.push_to_hub(os.path.basename(model_path), revision="chck_"+name)


In [ ]:
adfs

In [ ]:
from util import get_curriculum
word_count = lambda d: {"words": len(d["text"].split())}

In [ ]:
dataset_name = "loris3/stratified_10m_curriculum"

In [ ]:
curriculum = get_curriculum(dataset_name, "roberta_influence_incr_bins_lognorm.pt")

In [ ]:
dataset = load_dataset(dataset_name)["train"]
word_counts_dataset = dataset.map(word_count, num_proc=100)


wc = 0
wc_total = 0

dataset_indices_after_wich_to_checkpoint = []

indices = torch.cat(curriculum).flatten().tolist()
selected = word_counts_dataset.select(indices)

for i,row in enumerate(selected):

    wc += row["words"]
    wc_total += row["words"]
    if (wc >= 1_000_000 and wc_total <= 10_000_000):
        dataset_indices_after_wich_to_checkpoint.append(i)
        wc = 0
    elif (wc >= 10_000_000):
        dataset_indices_after_wich_to_checkpoint.append(i)
        wc = 0
dataset_indices_after_wich_to_checkpoint

In [ ]:
len(dataset_indices_after_wich_to_checkpoint)

In [ ]:
import math


effective_batch_size = 32 * 4 * 16
save_steps = [math.ceil(i / effective_batch_size) for i in dataset_indices_after_wich_to_checkpoint]
save_steps

In [ ]:
len(save_steps)